In [1]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.model_selection import train_test_split
import random
import tifffile as tiff

In [11]:
data_sign = 'Pavia'
train_num_per_class = 10
data_path_prefix = '../../data'
save_path = '../../data/%s/%s_split.mat' %(data_sign, data_sign)

In [7]:
def load_data(data_sign):
    if data_sign == "Indian":
        data = sio.loadmat('%s/Indian_pines_corrected.mat' % data_path_prefix)['indian_pines_corrected']
        labels = sio.loadmat('%s/Indian_pines_gt.mat' % data_path_prefix)['indian_pines_gt']
    elif data_sign == "Pavia":
        data = sio.loadmat('%s/PaviaU.mat' % data_path_prefix)['paviaU']
        labels = sio.loadmat('%s/PaviaU_gt.mat' % data_path_prefix)['paviaU_gt'] 
    elif data_sign == "Houston":
        data = sio.loadmat('%s/Houston.mat' % data_path_prefix)['img']
        labels = sio.loadmat('%s/Houston_gt.mat' % data_path_prefix)['Houston_gt']
    return data, labels

def gen(data_sign, train_num_per_class):
    data, labels = load_data(data_sign)
    h, w, c = data.shape
    class_num = labels.max()
    class2data = {}
    for i in range(h):
        for j in range(w):
            if labels[i,j] > 0:
                if labels[i, j] in class2data:
                    class2data[labels[i,j]].append([i, j])
                else:
                    class2data[labels[i,j]] = [[i,j]]

    TR = np.zeros_like(labels)
    TE = np.zeros_like(labels)
    for cl in range(class_num):
        class_index = cl + 1
        ll = class2data[class_index]
        all_index = list(range(len(ll)))
        select_train_index = set(random.sample(all_index, train_num_per_class))
        for index in select_train_index:
            item = ll[index]
            TR[item[0], item[1]] = class_index
    TE = labels - TR
    target = {}
    target['TE'] = TE
    target['TR'] = TR
    target['input'] = data
    return target



In [13]:
target = gen(data_sign, train_num_per_class)
sio.savemat(save_path, target)